In [85]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import gensim
from gensim.models.doc2vec import Doc2Vec
from nltk.tokenize import word_tokenize

import data_pre_processing as file_dpp

In [86]:
df_list = file_dpp.main_proc()
merged_game_sale_and_reviews = df_list[-1]
merged_game_sale_and_reviews

(16593, 2)
(16713, 2)
(5516, 2)


,name,review,game,platform,score,date,Name,Platform,Year_of_Release,Genre,...,EU_Sales,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating
0,LEVEL (Czech Republic),"Portal 2 is a masterpiece, a work of art that ...",portal 2,PC,100.0,"May 25, 2011",portal 2,PC,2011.0,Shooter,...,0.32,0.0,0.1,0.76,95.0,52.0,8.8,5999.0,Valve Software,E10+
1,GameCritics,So do we need Portal 2? Do I need it? Maybe no...,portal 2,PC,100.0,"May 8, 2011",portal 2,PC,2011.0,Shooter,...,0.32,0.0,0.1,0.76,95.0,52.0,8.8,5999.0,Valve Software,E10+
2,PC Games (Russia),Portal 2 exceeds every expectation. It has a s...,portal 2,PC,100.0,"May 6, 2011",portal 2,PC,2011.0,Shooter,...,0.32,0.0,0.1,0.76,95.0,52.0,8.8,5999.0,Valve Software,E10+
3,Adventure Gamers,"Like its predecessor, Portal 2 is not an adven...",portal 2,PC,100.0,"Apr 29, 2011",portal 2,PC,2011.0,Shooter,...,0.32,0.0,0.1,0.76,95.0,52.0,8.8,5999.0,Valve Software,E10+
4,Armchair Empire,"Pile on the ""Oh, yes!"" moments of solving some...",portal 2,PC,100.0,"Apr 28, 2011",portal 2,PC,2011.0,Shooter,...,0.32,0.0,0.1,0.76,95.0,52.0,8.8,5999.0,Valve Software,E10+
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
32774,XboxAddict,Despite the lackluster quality with some of it...,ride 2,XOne,70.0,"Mar 7, 2017",ride 2,XOne,2016.0,Racing,...,0.01,0.0,0.0,0.01,NaN,NaN,8.2,6.0,Milestone S.r.l,E
32775,ICXM,Ride 2 is a huge improvement over the original...,ride 2,XOne,70.0,"Feb 18, 2017",ride 2,XOne,2016.0,Racing,...,0.01,0.0,0.0,0.01,NaN,NaN,8.2,6.0,Milestone S.r.l,E
32776,Worth Playing,Ride 2 is certainly an improvement over the or...,ride 2,XOne,65.0,"Mar 9, 2017",ride 2,XOne,2016.0,Racing,...,0.01,0.0,0.0,0.01,NaN,NaN,8.2,6.0,Milestone S.r.l,E
32777,TrueAchievements,"Despite the addition of more bikes and tracks,...",ride 2,XOne,60.0,"Oct 13, 2016",ride 2,XOne,2016.0,Racing,...,0.01,0.0,0.0,0.01,NaN,NaN,8.2,6.0,Milestone S.r.l,E


# Embeddings section

Some parts of this section are from the article in the link below.

https://medium.com/@ermolushka/text-clusterization-using-python-and-doc2vec-8c499668fa61

The author demonstrates how to retrieve embeddings from the IMDB reviews. 

In [87]:
 
#Clean and tokenize the reviews
merged_game_sale_and_reviews['review_tokens'] = merged_game_sale_and_reviews['review'].str.lower()
merged_game_sale_and_reviews['review_tokens'] = merged_game_sale_and_reviews['review_tokens'].apply(word_tokenize)
merged_game_sale_and_reviews[['review','review_tokens']].head()

,review,review_tokens
0,"Portal 2 is a masterpiece, a work of art that ...","[portal, 2, is, a, masterpiece, ,, a, work, of..."
1,So do we need Portal 2? Do I need it? Maybe no...,"[so, do, we, need, portal, 2, ?, do, i, need, ..."
2,Portal 2 exceeds every expectation. It has a s...,"[portal, 2, exceeds, every, expectation, ., it..."
3,"Like its predecessor, Portal 2 is not an adven...","[like, its, predecessor, ,, portal, 2, is, not..."
4,"Pile on the ""Oh, yes!"" moments of solving some...","[pile, on, the, ``, oh, ,, yes, !, '', moments..."


In [88]:
merged_game_sale_and_reviews[merged_game_sale_and_reviews['game'].str.contains('counter')]

,name,review,game,platform,score,date,Name,Platform,Year_of_Release,Genre,...,JP_Sales,Other_Sales,Global_Sales,Critic_Score,Critic_Count,User_Score,User_Count,Developer,Rating,review_tokens


In [89]:
#Give tag numbers to each review
LabeledSentence1 = gensim.models.doc2vec.TaggedDocument
all_content_train = []
j=0
for em in merged_game_sale_and_reviews['review_tokens'].values:
    all_content_train.append(LabeledSentence1(em,[j]))
    j+=1
print('Number of texts processed: ', j)

Number of texts processed:  32779


In [90]:
print(len(all_content_train))
all_content_train[:5]

32779


[TaggedDocument(words=['portal', '2', 'is', 'a', 'masterpiece', ',', 'a', 'work', 'of', 'art', 'that', 'you', 'will', 'love', 'for', 'its', 'ingenious', 'story', 'pop-culture', ',', 'references', 'to', 'logical', 'puzzles', 'and', 'co-op', 'multiplayer', '.', 'definitely', 'a', 'candidate', 'for', 'game', 'of', 'the', 'year', '.', '[', 'issue', '#', '204', ']'], tags=[0]),
 TaggedDocument(words=['so', 'do', 'we', 'need', 'portal', '2', '?', 'do', 'i', 'need', 'it', '?', 'maybe', 'not', ',', 'but', 'i', "'m", 'sure', 'as', 'hell', 'glad', 'it', 'exists', '.', 'the', 'portal', 'aspect', 'has', 'probably', 'reached', 'its', 'zenith', 'in', 'portal', '2', ',', 'and', 'given', 'the', 'way', 'the', 'game', 'ends', 'i', 'do', "n't", 'think', 'there', "'s", 'much', 'room', 'for', 'a', 'portal', '3', '.'], tags=[1]),
 TaggedDocument(words=['portal', '2', 'exceeds', 'every', 'expectation', '.', 'it', 'has', 'a', 'smart', 'story', ',', 'hilarious', 'dialogs', 'and', 'the', 'most', 'refined', 'mul

In [91]:
#Create a function to construct document embeddings
def get_doc2vec(tagged_doc, param_list):
    vector_size = param_list[0]
    window = param_list[1]
    min_count = param_list[2]
    negative = param_list[3]
    epoch = param_list[4]
    
    d2v_model = Doc2Vec(tagged_doc, vector_size=vector_size, window=window, min_count=min_count, workers=7, negative=negative)
    d2v_model.train(tagged_doc, total_examples=d2v_model.corpus_count, epochs=epoch)
    
    return d2v_model

In [193]:
#Parameters for Doc2Vec
vector_size = 50
window = 5
min_count = 1
negative = 0
epoch = 10

param_list = [vector_size, window, min_count, negative, epoch]

#Get Doc2Vec model from the given parameters
d2v_model = get_doc2vec(all_content_train, param_list)

In [194]:
#Get input game name from the user and clean the input
input_game_name = input('Enter game name: ')
input_game_name = input_game_name.lower()
input_game_name = input_game_name.strip()
print(input_game_name)

fallout


In [195]:
#Get input game score from the user, clean it, and change the format to float
input_game_score = input('Enter game score (0.0-10.0): ')
input_game_score = input_game_score.strip()
input_game_score = float(input_game_score)
print(input_game_score)

10.0


In [196]:
#Filter the df to get only the rows that contain the input game name 
filtered_df = merged_game_sale_and_reviews[merged_game_sale_and_reviews['game'].str.contains(input_game_name)]
filtered_df.shape

(137, 23)

In [197]:
#Get the closest User Score from the filtered_df
unique_user_score = filtered_df['User_Score'].unique()
print(unique_user_score)
dif_array = np.abs(unique_user_score - input_game_score)
closest_user_score = unique_user_score[dif_array.argmin()]
closest_user_score

[6.2 6.5 5.4]


6.5

In [198]:
#Filter the df again to get the rows that contain the closest game score
filtered_df = filtered_df[filtered_df['User_Score']==closest_user_score]
filtered_df.shape

(58, 23)

In [199]:
#Get the following embeddings from the filtered_df indexes 
filtered_d2v = d2v_model.dv[filtered_df.index]
filtered_d2v.shape

(58, 50)

In [200]:
#Average the filtered embeddings to get an average vector in the vector space
avg_filtered_d2v = np.average(filtered_d2v, axis=0)
avg_filtered_d2v.shape

(50,)

In [201]:
#Get the most 10 similar embeddings (compared to the avg filtered embedding) 
similar_reviews = d2v_model.dv.most_similar([avg_filtered_d2v])
similar_reviews

[(10682, 0.5659517645835876),
 (19068, 0.4866109788417816),
 (14410, 0.48263850808143616),
 (27087, 0.4796906113624573),
 (21602, 0.46791911125183105),
 (9827, 0.46771469712257385),
 (3936, 0.46480968594551086),
 (14743, 0.461399108171463),
 (7642, 0.4575643837451935),
 (19064, 0.4553277790546417)]

In [202]:
#Print out the inputs and the recommended games
game_idx = [i[0] for i in similar_reviews]
print(game_idx)
game_similarity = [i[1] for i in similar_reviews]
print(game_similarity)
recommended_games = merged_game_sale_and_reviews.loc[game_idx,:]['game'].unique()

print(' ')
print('Input game: ' + input_game_name)
print('Input user score: ' + str(input_game_score))
print('Similarity - Recommended games')
for i, j in zip(recommended_games, game_similarity):
    print(str(j) + ' - ' + i)

[10682, 19068, 14410, 27087, 21602, 9827, 3936, 14743, 7642, 19064]
[0.5659517645835876, 0.4866109788417816, 0.48263850808143616, 0.4796906113624573, 0.46791911125183105, 0.46771469712257385, 0.46480968594551086, 0.461399108171463, 0.4575643837451935, 0.4553277790546417]
 
Input game: fallout
Input user score: 10.0
Similarity - Recommended games
0.5659517645835876 - castlevania: lords of shadow - mirror of fate
0.4866109788417816 - fallout 4
0.48263850808143616 - danganronpa: trigger happy havoc
0.4796906113624573 - watch dogs 2
0.46791911125183105 - etrian mystery dungeon
0.46771469712257385 - the wonderful 101
0.46480968594551086 - bust-a-move universe
0.461399108171463 - wolfenstein: the new order
0.4575643837451935 - little deviants


In [203]:
# Function to add a column to Doc2Vec embeddings
def append_col_to_d2v(d2v_model,added_col):
    '''
    
    :param d2v_model: Doc2Vec embeddings 
    :param added_col: A column that the user want to add to the embedding, should be a DataFrame, series, or numpy array, and has 1D
    :return: A new Doc2Vec embeddings that is already added the additional column 
    '''
    added_col_arr = np.array(added_col).reshape(-1,1)
    d2v_model.dv.vectors = np.append(d2v_model.dv.vectors,added_col_arr,axis=1)
    
    return d2v_model

In [204]:
# Test to add one columns to the Doc2Vec embeddings

# At first, the embeddings has this shape
print('Shape of the embeddings before appending a new column: ' + str(d2v_model.dv.vectors.shape))

# Initiate an array that contains only 18 in it and has one dimension equal to the number of reviews 
test_array = np.ones(len(all_content_train)).reshape(-1,1)*18

# Call the add column function and print the dimension after append the additional column
new_d2v_model = append_col_to_d2v(d2v_model, test_array)
print('The new shape of the embeddings after appending a new column: ' + str(new_d2v_model.dv.vectors.shape))

Shape of the embeddings before appending a new column: (32779, 50)
[[18.]
 [18.]
 [18.]
 ...
 [18.]
 [18.]
 [18.]]
The new shape of the embeddings after appending a new column: (32779, 51)


In [206]:
# You can see that there is 18 in the last position of each vector
new_d2v_model.dv.vectors


array([[-1.04616312e-02, -1.19582554e-02, -1.97615102e-02, ...,
         1.08264545e-02, -1.60051435e-02,  1.80000000e+01],
       [-4.23970213e-03, -1.91631319e-04, -1.32775549e-02, ...,
         1.77903362e-02,  4.70251078e-03,  1.80000000e+01],
       [-1.11207394e-02, -6.36106962e-03, -1.19417142e-02, ...,
         5.98902698e-04, -9.41679440e-03,  1.80000000e+01],
       ...,
       [-1.60153247e-02,  4.59572300e-03,  1.27786165e-03, ...,
         1.07678510e-02, -1.08869886e-02,  1.80000000e+01],
       [-2.06432818e-03,  1.47844316e-03,  1.86165329e-02, ...,
        -4.82443348e-03,  1.93362050e-02,  1.80000000e+01],
       [-2.05020420e-03, -1.68271214e-02, -1.77463051e-03, ...,
         7.87022617e-03, -6.47610659e-03,  1.80000000e+01]])